In [1]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np

In [2]:
from szp_funcs import print_df, load_month, custom_create_res, months_year

In [13]:
months = months_year[:4]
job = ['Юрисконсульт', 'Ведущий юрисконсульт']

In [5]:
res = custom_create_res(months, load_month)

In [8]:
def re_szp(row, months):
    active_months = 0
    fot = 0
    for month in months:
        if row['day_' + month] == 1 and row['stv_' + month] >= 1 and row['status_' + month] == row['status_pref_' + month] and row['status_' + month] == 'Работа':
            fot += row['sum_' + month]
            active_months += 1
    if active_months == 0:
        return np.nan
    return fot / active_months

In [9]:
def re_szp_job(row, months, jobs):
    active_months = 0
    fot = 0
    for month in months:
        if row['job_' + month] in jobs and row['day_' + month] == 1 and row['stv_' + month] >= 1 and row['status_' + month] == row['status_pref_' + month] and row['status_' + month] == 'Работа':
            fot += row['sum_' + month]
            active_months += 1
    if active_months == 0:
        return np.nan
    return fot / active_months

In [10]:
def re_fot(row, months):
    fot = 0
    for month in months:
        fot += row['sum_' + month]
    if fot == 0:
        return np.nan
    return fot

In [11]:
def re_fot_job(row, months, jobs):
    fot = 0
    for month in months:
        if row['job_' + month] in jobs:
            fot += row['sum_' + month]
    if fot == 0:
        return np.nan
    return fot

In [14]:
res['fot_job'] = res.apply(lambda row: re_fot_job(row, months, job), axis=1)
res['szp_job'] = res.apply(lambda row: re_szp_job(row, months, job), axis=1)

In [16]:
ans = res[['inn', 'fot_job', 'szp_job', 'stv_' + months[-1], 'snils']][res['job_' + months[-1]].isin(job)].groupby('inn').agg({
    'fot_job': 'sum',
    'szp_job': 'mean',
    'stv_' + months[-1]: 'sum',
    'snils': 'nunique'
})

In [28]:
res[(res['job_' + months[-1]].isin(job)) & (res.inn == 7737078033)]#[['job_' + month for month in months]]

,inn,snils,sum_jan,job_jan,day_jan,type_jan,stv_jan,status_jan,status_pref_jan,sum_feb,...,job_apr,day_apr,type_apr,stv_apr,status_apr,status_pref_apr,wday_apr,nday_apr,fot_job,szp_job
172134,7737078033,153-358-303 56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29763.9,...,Юрисконсульт,1.0,Основное место работы,1.0,Работа,Работа,21.0,21.0,139144.62,54690.36


In [17]:
print_df(ans, job[0] + ' за ' + months[-1])